In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Instale a biblioteca necessária para conversão de coordenadas
!pip install mercantile

In [ ]:
import os
import requests
import mercantile
import time
from concurrent.futures import ThreadPoolExecutor

In [ ]:
# --- PASSO 1: DEFINIR A ÁREA DE INTERESSE E O ZOOM ---

# Bounding Box (Caixa Delimitadora) para Linhares, ES
# Formato: (longitude_oeste, latitude_sul, longitude_leste, latitude_norte)
linhares_bbox = (-40.09363, -19.42839, -40.00153, -19.30013)


# Nível de Zoom que você quer baixar
zoom_level = 20

# URL base do serviço de tiles
base_url = "https://ortofotos.linhares.es.gov.br/SEDE/2022"

# Pasta local onde os arquivos serão salvos
output_dir = "/content/ortofotos_linhares"


In [ ]:
# --- PASSO 2: CONVERTER BOUNDING BOX PARA TILES ---

# A biblioteca mercantile calcula quais tiles cobrem a nossa bounding box no zoom especificado
# O resultado é uma lista de todos os tiles (x, y, z)
tiles = list(mercantile.tiles(west=linhares_bbox[0],
                               south=linhares_bbox[1],
                               east=linhares_bbox[2],
                               north=linhares_bbox[3],
                               zooms=[zoom_level]))

print(f"Total de tiles a serem verificados para a área: {len(tiles)}")

if not tiles:
    print("Nenhum tile encontrado para a BBox e zoom especificados. Verifique as coordenadas.")
else:
    # --- PASSO 3: FUNÇÃO DE DOWNLOAD ---

    def download_tile(tile):
        """Função para baixar um único tile."""
        x, y, z = tile.x, tile.y, tile.z

        # Monta a URL completa do tile
        tile_url = f"{base_url}/{z}/{x}/{y}.png"

        # Monta o caminho local para salvar o arquivo, replicando a estrutura do servidor
        save_path_dir = os.path.join(output_dir, str(z), str(x))
        os.makedirs(save_path_dir, exist_ok=True)
        save_path_file = os.path.join(save_path_dir, f"{y}.png")

        # Verifica se o arquivo já existe para não baixar de novo
        if os.path.exists(save_path_file):
            # print(f"Tile {z}/{x}/{y} já existe. Pulando.")
            return f"EXISTE: {tile_url}"

        try:
            # Faz a requisição para baixar a imagem
            response = requests.get(tile_url, timeout=10)

            # Se a resposta for 200 (OK), salva o arquivo
            if response.status_code == 200:
                with open(save_path_file, 'wb') as f:
                    f.write(response.content)
                return f"BAIXADO: {tile_url}"
            else:
                return f"FALHA {response.status_code}: {tile_url}"
        except requests.exceptions.RequestException as e:
            return f"ERRO DE CONEXÃO: {tile_url} ({e})"

    # --- PASSO 4: EXECUTAR O DOWNLOAD EM PARALELO ---

    # Usar ThreadPoolExecutor acelera o processo ao fazer vários downloads simultaneamente
    # O número de 'workers' pode ser ajustado
    with ThreadPoolExecutor(max_workers=10) as executor:
        results = list(executor.map(download_tile, tiles))

    # Imprime um resumo do que foi feito
    baixados = sum(1 for r in results if r.startswith("BAIXADO"))
    falhas = sum(1 for r in results if r.startswith("FALHA"))
    erros = sum(1 for r in results if r.startswith("ERRO"))
    existentes = sum(1 for r in results if r.startswith("EXISTE"))

    print("\n--- Download Concluído! ---")
    print(f"Tiles baixados com sucesso: {baixados}")
    print(f"Tiles já existentes: {existentes}")
    print(f"Tiles não encontrados no servidor (Falha 404, etc.): {falhas}")
    print(f"Erros de conexão: {erros}")
    print(f"Imagens salvas em: {output_dir}")